# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [41]:
# import libraries
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
#


# Stop Words
# Combine the steps you learned so far to normalize, tokenize, and remove stop words from the text below.


# import statements
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('stopwords')
#
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import GridSearchCV
from sklearn.multioutput import MultiOutputClassifier


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Thomas\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Thomas\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [42]:
databasename = '../data/DisasterResponse.db'
startupdatabase = 'sqlite:///' + databasename
tablename = 'disastertweets'
modelname = '../model/DisasterResponse.pkl'

In [43]:
# load data from database
#engine = create_engine('sqlite:///InsertDatabaseName.db')
#engine = create_engine(startupdatabase)
df = pd.read_sql_table(tablename, startupdatabase)
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [44]:
X = df.message.values
y = df.loc[:,'related':]
print (df.shape)
print (X.shape)
print (y.shape)

(26180, 40)
(26180,)
(26180, 36)


### 2. Write a tokenization function to process your text data

In [45]:
def tokenize(text):
    #detected_urls = re.findall(url_regex, text)
    #for url in detected_urls:
    #    text = text.replace(url, "urlplaceholder")
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    tokens = word_tokenize(text)
    # print(tokens)
    tokens = [w for w in tokens if w not in stopwords.words("english")]
    # print(tokens)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens


In [46]:
ct = tokenize('UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs ??? supplies. ? desperately.')
print (ct)


['un', 'report', 'leogane', '80', '90', 'destroyed', 'hospital', 'st', 'croix', 'functioning', 'need', 'supply', 'desperately']


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [47]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [48]:
# already done and cleaned X, y = load_data()
X_train, X_test, y_train, y_test = train_test_split(X, y)
# train classifier
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at...
                                                                        ccp_alpha=0.0,
                                                                        class_weight=None,
                                                                        criterion='gini',
                                                                   

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [49]:
# predict on test data
y_pred = pipeline.predict(X_test)

In [50]:
#print(y_pred)
#print(y_test)

#print (X_test)
print (y_test.loc[:,'related'].values)
print (y_pred[:,0])

[0 1 1 ... 0 1 1]
[1 1 1 ... 1 1 1]


In [51]:
for s in y_test.columns:
    u = classification_report(y_test.loc[:,s].values,y_pred[:,0])
    print(s, u)

related               precision    recall  f1-score   support

           0       0.71      0.42      0.53      1547
           1       0.84      0.94      0.89      4950
           2       0.22      0.38      0.28        48

    accuracy                           0.81      6545
   macro avg       0.59      0.58      0.56      6545
weighted avg       0.80      0.81      0.80      6545



C:\Users\Thomas\Anaconda3\envs\Udacity\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


request               precision    recall  f1-score   support

           0       0.97      0.16      0.28      5430
           1       0.19      0.97      0.32      1115
           2       0.00      0.00      0.00         0

    accuracy                           0.30      6545
   macro avg       0.39      0.38      0.20      6545
weighted avg       0.83      0.30      0.29      6545

offer               precision    recall  f1-score   support

           0       1.00      0.14      0.24      6519
           1       0.00      0.92      0.01        26
           2       0.00      0.00      0.00         0

    accuracy                           0.14      6545
   macro avg       0.33      0.35      0.08      6545
weighted avg       0.99      0.14      0.24      6545

aid_related               precision    recall  f1-score   support

           0       0.95      0.22      0.36      3865
           1       0.47      0.98      0.64      2680
           2       0.00      0.00      0.00      


hospitals               precision    recall  f1-score   support

           0       1.00      0.14      0.25      6487
           1       0.01      1.00      0.02        58
           2       0.00      0.00      0.00         0

    accuracy                           0.15      6545
   macro avg       0.34      0.38      0.09      6545
weighted avg       0.99      0.15      0.24      6545

shops               precision    recall  f1-score   support

           0       1.00      0.14      0.25      6515
           1       0.01      1.00      0.01        30
           2       0.00      0.00      0.00         0

    accuracy                           0.14      6545
   macro avg       0.34      0.38      0.09      6545
weighted avg       1.00      0.14      0.24      6545

aid_centers               precision    recall  f1-score   support

           0       1.00      0.14      0.25      6466
           1       0.01      0.99      0.03        79
           2       0.00      0.00      0.00   

### 6. Improve your model
Use grid search to find better parameters. 

In [54]:
from sklearn.model_selection import GridSearchCV

pipeline.get_params().keys()



dict_keys(['memory', 'steps', 'verbose', 'vect', 'tfidf', 'clf', 'vect__analyzer', 'vect__binary', 'vect__decode_error', 'vect__dtype', 'vect__encoding', 'vect__input', 'vect__lowercase', 'vect__max_df', 'vect__max_features', 'vect__min_df', 'vect__ngram_range', 'vect__preprocessor', 'vect__stop_words', 'vect__strip_accents', 'vect__token_pattern', 'vect__tokenizer', 'vect__vocabulary', 'tfidf__norm', 'tfidf__smooth_idf', 'tfidf__sublinear_tf', 'tfidf__use_idf', 'clf__estimator__bootstrap', 'clf__estimator__ccp_alpha', 'clf__estimator__class_weight', 'clf__estimator__criterion', 'clf__estimator__max_depth', 'clf__estimator__max_features', 'clf__estimator__max_leaf_nodes', 'clf__estimator__max_samples', 'clf__estimator__min_impurity_decrease', 'clf__estimator__min_impurity_split', 'clf__estimator__min_samples_leaf', 'clf__estimator__min_samples_split', 'clf__estimator__min_weight_fraction_leaf', 'clf__estimator__n_estimators', 'clf__estimator__n_jobs', 'clf__estimator__oob_score', 'clf_

In [55]:
parameters = {
        'clf__estimator__n_estimators': [50, 100, 200],
        'clf__estimator__min_samples_split': [2, 3, 4],
    }


cv = GridSearchCV(pipeline, param_grid=parameters)

cv.fit(X_train, y_train)
y_pred = cv.predict(X_test)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
for s in y_test.columns:
    u = classification_report(y_test.loc[:,s].values,y_pred[:,0])
    print(s, u)

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [ ]:
from sklearn.externals import joblib

joblib.dump(cv,modelname)
print("#----------------------- pickle export done")

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.